# Predict

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

### Imports

In [1]:
# Imports
import os
import warnings
import tools
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from matplotlib import pyplot
import numpy as np
from sklearn.metrics import confusion_matrix
import sklearn
import wandb

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load Data

In [2]:
# Root CSV files directory
dirname = "./data/absolute/3D/"  


# Load data and print summary, if desired
x_train, x_val, x_test, y_train, y_val, y_test, labels = tools.load_from(dirname, verbose=True) 

Amount Datasets by word total:
Computer :  58;  Deutschland :  67;  Haben :  71;  Hallo :  62;  Mainz :  67;  Software :  68;  Welt :  68;  du :  70;  ich :  38;  unser :  67;  zeigen :  70;   
Amount Datasets by word training:
Computer :  35;  Deutschland :  40;  Haben :  42;  Hallo :  37;  Mainz :  40;  Software :  41;  Welt :  41;  du :  42;  ich :  23;  unser :  40;  zeigen :  42;   
Amount Datasets by word validiation:
Computer :  12;  Deutschland :  13;  Haben :  14;  Hallo :  13;  Mainz :  14;  Software :  13;  Welt :  13;  du :  14;  ich :  8;  unser :  13;  zeigen :  14;   
Amount Datasets by word test:
Computer :  11;  Deutschland :  14;  Haben :  15;  Hallo :  12;  Mainz :  13;  Software :  14;  Welt :  14;  du :  14;  ich :  7;  unser :  14;  zeigen :  14;   
Distribution of data:
Amount total: 706
Amount training: 423
Amount validiation: 141
Amount test: 142
Tokens:
{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8, 'uns

In [3]:
with open('tokens_json.txt', 'r') as outfile:
    json_ex = outfile.read()
    

tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_ex)
print(tokenizer.word_index)
token_labels = {y:x for x,y in tokenizer.word_index.items()}

{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8, 'unser': 9, 'welt': 10, 'zeigen': 11}


In [4]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


In [5]:
model = tf.keras.models.load_model('model-best_3d.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           49408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 64)           33024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 128)          98816     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 12)                780       
Total params: 264,460
Trainable params: 264,460
Non-trainable params: 0
__________________________________________________

### Evaluate

In [6]:
eval = model.evaluate(x_test, y_test, verbose=2)

142/142 - 1s - loss: 0.5466 - accuracy: 0.8521 - precision: 0.8551 - recall: 0.8310


In [7]:
predict = model.predict(x_test)
# y_pred = np.array([np.argmax(pred) for pred in predict])
y_pred_integer = np.argmax(predict, axis=1)
y_test_integer = np.argmax(y_test, axis=1)

y_pred_name = ([token_labels[p] for p in y_pred_integer])
y_test_name = ([token_labels[p] for p in y_test_integer])


In [8]:
y_pred_name

['software',
 'welt',
 'unser',
 'software',
 'zeigen',
 'mainz',
 'unser',
 'zeigen',
 'welt',
 'hallo',
 'deutschland',
 'computer',
 'welt',
 'deutschland',
 'welt',
 'welt',
 'mainz',
 'computer',
 'unser',
 'hallo',
 'ich',
 'haben',
 'zeigen',
 'mainz',
 'software',
 'software',
 'deutschland',
 'welt',
 'hallo',
 'software',
 'deutschland',
 'haben',
 'du',
 'unser',
 'software',
 'haben',
 'unser',
 'zeigen',
 'computer',
 'software',
 'haben',
 'hallo',
 'zeigen',
 'haben',
 'du',
 'zeigen',
 'deutschland',
 'haben',
 'du',
 'haben',
 'zeigen',
 'computer',
 'du',
 'deutschland',
 'software',
 'computer',
 'hallo',
 'zeigen',
 'mainz',
 'deutschland',
 'haben',
 'welt',
 'zeigen',
 'unser',
 'unser',
 'unser',
 'unser',
 'computer',
 'du',
 'haben',
 'haben',
 'welt',
 'software',
 'software',
 'unser',
 'unser',
 'zeigen',
 'haben',
 'mainz',
 'hallo',
 'du',
 'haben',
 'software',
 'software',
 'deutschland',
 'welt',
 'du',
 'deutschland',
 'haben',
 'unser',
 'du',
 'welt'

In [9]:
y_test_name 

['software',
 'welt',
 'unser',
 'welt',
 'zeigen',
 'haben',
 'hallo',
 'zeigen',
 'welt',
 'hallo',
 'deutschland',
 'computer',
 'welt',
 'deutschland',
 'software',
 'welt',
 'mainz',
 'computer',
 'unser',
 'hallo',
 'ich',
 'haben',
 'zeigen',
 'ich',
 'software',
 'software',
 'deutschland',
 'welt',
 'hallo',
 'software',
 'deutschland',
 'haben',
 'unser',
 'unser',
 'software',
 'mainz',
 'hallo',
 'zeigen',
 'computer',
 'software',
 'haben',
 'hallo',
 'zeigen',
 'haben',
 'du',
 'zeigen',
 'deutschland',
 'haben',
 'du',
 'haben',
 'zeigen',
 'computer',
 'du',
 'deutschland',
 'software',
 'computer',
 'hallo',
 'zeigen',
 'mainz',
 'welt',
 'haben',
 'welt',
 'zeigen',
 'unser',
 'unser',
 'unser',
 'unser',
 'computer',
 'du',
 'haben',
 'haben',
 'welt',
 'welt',
 'software',
 'unser',
 'unser',
 'zeigen',
 'haben',
 'mainz',
 'hallo',
 'du',
 'haben',
 'software',
 'software',
 'deutschland',
 'software',
 'zeigen',
 'mainz',
 'mainz',
 'unser',
 'du',
 'welt',
 'hall

In [ ]:
wandb.init(project="Test", name="confusion")

In [ ]:
# Confusion Matrix
#sklearn.metrics.plot_confusion_matrix(y_test_name, y_pred_name)
wandb.sklearn.plot_confusion_matrix(y_test_name, y_pred_name)

# TFLITE

In [61]:
x_test=np.array(x_test,dtype=np.float32)


print(type(x_train[0][0][0]))
print(type(x_test[0][0][0]))


<class 'numpy.float32'>
<class 'numpy.float32'>


In [62]:
import numpy as np
import tensorflow as tf
from time import time

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition_3D.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)


input_shape = input_details[0]['shape']
time1= time()
input_data = [1]
predictions = []

input_details[0]["index"]

for x_pred in x_test:
    
    input_data[0] = x_pred
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
    
predictions = [token_labels[p] for p in predictions]

[{'name': 'lstm_input', 'index': 0, 'shape': array([  1, 100, 128]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [72]:
for i in range(0, len(predictions)):
    print('keras: ',y_pred_name[i],  predictions[i])

keras:  software software
keras:  welt welt
keras:  unser unser
keras:  software software
keras:  zeigen zeigen
keras:  mainz mainz
keras:  unser unser
keras:  zeigen zeigen
keras:  welt welt
keras:  hallo hallo
keras:  deutschland deutschland
keras:  computer computer
keras:  welt welt
keras:  deutschland deutschland
keras:  welt welt
keras:  welt welt
keras:  mainz mainz
keras:  computer computer
keras:  unser unser
keras:  hallo hallo
keras:  ich ich
keras:  haben haben
keras:  zeigen zeigen
keras:  mainz mainz
keras:  software software
keras:  software software
keras:  deutschland deutschland
keras:  welt welt
keras:  hallo hallo
keras:  software software
keras:  deutschland deutschland
keras:  haben haben
keras:  du du
keras:  unser unser
keras:  software software
keras:  haben haben
keras:  unser unser
keras:  zeigen zeigen
keras:  computer computer
keras:  software software
keras:  haben haben
keras:  hallo hallo
keras:  zeigen zeigen
keras:  haben haben
keras:  du du
keras:  ze

In [ ]:
print("Predictions:", predictions)
time2= time()
dauer = time2 - time1
print("Anzahl Samples: ", len(predictions))
print("Predictiondauer: ", dauer)
print("Samples pro Sekunde:", (len(predictions)/dauer))

In [11]:
print(input_data)

[array([[0.497013, 0.212492, 0.      , ..., 0.5     , 0.5     , 0.5     ],
       [0.495045, 0.21436 , 0.      , ..., 0.5     , 0.5     , 0.5     ],
       [0.49323 , 0.211844, 0.      , ..., 0.5     , 0.5     , 0.5     ],
       ...,
       [0.5     , 0.5     , 0.5     , ..., 0.5     , 0.5     , 0.5     ],
       [0.5     , 0.5     , 0.5     , ..., 0.5     , 0.5     , 0.5     ],
       [0.5     , 0.5     , 0.5     , ..., 0.5     , 0.5     , 0.5     ]])]
